In [2]:
import _init_paths
import nengo_dl
import matplotlib.pyplot as plt
import numpy as np
import pathlib
import pickle

from configs.exp_configs import tf_exp_cfg as exp_cfg, nengo_dl_cfg as ngo_cfg
from utils.base_utils.data_prep_utils import get_batches_of_cifar_data
from utils.base_utils.exp_utils import collect_sim_data_spikes
from utils.nengo_dl_utils import get_nengo_dl_model
from utils.consts.dir_consts import EXP_OTPT_DIR
from utils.consts.exp_consts import SEED

/home/rgaurav/miniconda3/envs/nengo-tf/lib/python3.7/site-packages/nengo_dl/converter.py:360: UserWarning: Layer '<class 'tensorflow.python.keras.layers.normalization_v2.BatchNormalization'>' already has a converter. Overwriting.
  "Layer '%s' already has a converter. Overwriting." % keras_layer


# Get the batched Test Data, the Nengo-DL Model

In [3]:
batches = get_batches_of_cifar_data(ngo_cfg, is_test=True)
nengo_model, nengo_probes_obj_lst = get_nengo_dl_model(
    (32, 32, 3), exp_cfg, ngo_cfg)

/home/rgaurav/miniconda3/envs/nengo-tf/lib/python3.7/site-packages/nengo_dl/converter.py:555: UserWarning: Activation type <function softmax at 0x2b97f42668c0> does not have a native Nengo equivalent; falling back to a TensorNode
  "falling back to a TensorNode" % activation



# Create the Simulator and do inference

In [4]:
ndl_model_otpt = []
ndl_model_spks = []

with nengo_model.net:
  nengo_dl.configure_settings(stateful=False)

with nengo_dl.Simulator(
  nengo_model.net, minibatch_size=ngo_cfg["test_batch_size"], progress_bar=True, seed=SEED) as sim:
  nengo_input = nengo_probes_obj_lst[0]
  nengo_output = nengo_probes_obj_lst[-1]
  batch_num=1
  for batch in batches:
    sim_data = sim.predict_on_batch({nengo_input: batch[0]})
    for y_true, y_pred in zip(batch[1], sim_data[nengo_output]):
      ndl_model_otpt.append((np.argmax([y_true]), np.argmax(y_pred[-1]), y_pred))
    if batch_num == 1: # Collect only for one batch, else memory consumption is too high.
      mdl_spks = collect_sim_data_spikes(nengo_probes_obj_lst, sim_data)
      ndl_model_spks.extend(mdl_spks)
    print("Batch: %s Done." % batch_num)
    batch_num += 1

Build finished in 0:00:00                                                      
Optimization finished in 0:00:00                                               
Construction finished in 0:00:02                                               
Batch: 1 Done.


In [6]:
pickle.dump({"acc_res": ndl_model_otpt, "spk_res": ndl_model_spks}, 
            open(ngo_cfg["ndl_res_otpt_dir"]+"ndl_%s_results_sfr_%s_nstps_%s.p" 
                % (exp_cfg["tf_model"]["name"], ngo_cfg["sfr"], ngo_cfg["n_steps"]), "wb"))

In [4]:
acc = 0
for y_true, y_pred, _ in ndl_model_otpt:
  if y_true == y_pred:
    acc += 1

print(acc/10000)

0.6383


In [5]:
# SFR = 1, N_STEPS=80, ACC: 10% random (No layer spikes)
# SFR = 50, N_STEPS=80, ACC: 42.1 (output layer starts spiking after 70ms)
# SFR = 100, N_STEPS=80, ACC: 60.73 (output layer starts spiking after 50ms)
# SFR = 200, N_STEPS=80, ACC: 63.85 (output layer starts spiking after 35ms)
# SFR = 250, N_STEPS=80, ACC: 64.56 (output layer starts spiking after 30ms)
# SFR = 600, N_STEPS=80, ACC: 64.98 (output layer starts spiking after 20ms)


# SFR = 250, N_STEPS=60, ACC: 63.85 (Same spiking pattern as N_STEPS=80)
# SFR = 250, N_STEPS=55, ACC: 62.21 (Same spiking pattern as N_STEPS=80)
# SFR = 250, N_STEPS=50, ACC: 60.02 (Same spiking pattern as N_STEPS=80)
# SFR = 250, N_STEPS=40, ACC: 40.87 (Same spiking pattern as N_STEPS=80)


# As you lower down the SFR (and keep the N_STEPS fixed), it takes longer for the output layer to spike.
# 

acc = 0
for y_true, y_pred, _ in ndl_model_otpt:
  if y_true == y_pred:
    acc += 1

print(acc/10000)

0.6383
